In [1]:
# pos tagging for fake news statement
## use pos-tagging to build features
from pycorenlp import StanfordCoreNLP
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn import metrics
from sklearn.svm import SVC
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
import pandas as pd
import numpy as np

# import libraries

import util
from nlp_util import NLP_Task
import preprocessing
import importlib

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [111]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [2]:
# load files
tr_file, va_file, te_file = util.load_files()

In [6]:
nlpTask = NLP_Task()

NLP_Task ready to use.


In [3]:
tr_dict = util.tsv_to_dict(tsv_file=tr_file)

In [44]:
va_dict = util.tsv_to_dict(tsv_file=va_file)

In [4]:
corpus = []
for statement in tr_dict['statement']:
    review = re.sub('[^a-zA-Z]',' ',statement)
    review = review.lower()
    review = review.split()
    review = ''.join(review)
    corpus.append(review)

In [9]:
importlib.reload(preprocessing)
unigram_pos, bigrams_pos, trigram_pos = preprocessing.extract_POS(statements=tr_dict['statement'])

NLP_Task ready to use.
Extracting POS Tags
Stringify
Finished


In [144]:
bigrams_pos

['<s>_VBZ VBZ_DT DT_NNPS NNPS_VBP VBP_JJ JJ_NN NN_VBZ VBZ_JJ JJ_NNS NNS_IN IN_NN',
 '<s>_WRB WRB_VBD VBD_DT DT_NN NN_IN IN_NN NN_VB VB_. ._<s> <s>_PRP PRP_VBD VBD_WRB WRB_JJ JJ_NN NN_VBD VBD_RP RP_WDT WDT_VBD VBD_TO TO_VB VB_IN IN_-LRB- -LRB-_NNP NNP_NNP NNP_NNP NNP_-RRB- -RRB-_NNP NNP_NN',
 '<s>_NNP NNP_NNP NNP_VBZ VBZ_IN IN_NNP NNP_NNP NNP_`` ``_IN IN_VBG VBG_TO TO_VB VB_NNP NNP_NNP NNP_DT DT_NN NN_IN IN_DT DT_NN NN_IN IN_NNP NNP_.',
 '<s>_NNP NNP_NN NN_NN NN_NN NN_VBZ VBZ_JJ JJ_TO TO_VB VB_JJ JJ_NN NN_NN NN_NNS',
 '<s>_DT DT_JJ JJ_NN NN_VBD VBD_IN IN_DT DT_NN NN_IN IN_PRP$ PRP$_NN',
 '<s>_DT DT_NNP NNP_NNPS NNPS_VBP VBP_VBN VBN_RBR RBR_JJ JJ_NNS NNS_IN IN_DT DT_JJ JJ_CD CD_NNS NNS_IN IN_DT DT_JJ JJ_NN NN_IN IN_JJ JJ_-LRB- -LRB-_NN NN_-RRB- -RRB-_NN NN_VBD VBD_IN IN_DT DT_JJ JJ_CD CD_NNS',
 '<s>_NNP NNP_NNP NNP_VBZ VBZ_RB RB_VBN VBN_IN IN_DT DT_NN NN_PRP PRP_VBZ VBZ_IN IN_NNS NNS_RB',
 '<s>_PRP PRP_VBP VBP_DT DT_JJ JJ_NN NN_IN IN_DT DT_NN NN_WP WP_VBZ VBZ_VBN VBN_RB RB_RB RB_JJ JJ_NN

In [45]:
unigram_pos_va, bigrams_pos_va, trigram_pos_va = preprocessing.extract_POS(statements=va_dict['statement'])

NLP_Task ready to use.
Extracting POS Tags
Stringify
Finished


In [137]:
## use countVectorizor to build postag output as features for training ML model
## uni_vocabulary: vocabulary=['cd','jjr','vb','jjs','nnp','in','vbp','to','rb','vbg','md','jj','dt']
## bi_vocabulary: vocabulary=['NNP NNP', 'IN DT', 'JJR IN', 'CD NN', 'IN CD', 'CD NNS', 'DT JJS']
## tri_vocabulary: ['NNP NNP NNP','CD NN IN','VBZ NNP NNP','IN DT NN','IN DT JJ','NN IN DT','<s> VBZ NNP','JJR IN CD','NNP NNP VBD','DT JJ CD','NNS IN DT']

cv_uni = CountVectorizer()
cv_bi = CountVectorizer()
cv_tri = CountVectorizer()

pos_uni_feats = cv_uni.fit_transform(unigram_pos).toarray()
pos_bi_feats = cv_bi.fit_transform(bigrams_pos).toarray()
pos_tri_feats = cv_tri.fit_transform(trigram_pos).toarray()

In [138]:
cv_uni_va = CountVectorizer(vocabulary=cv_uni.get_feature_names())
cv_bi_va = CountVectorizer(vocabulary=cv_bi.get_feature_names())
cv_tri_va = CountVectorizer(vocabulary=cv_tri.get_feature_names())

pos_uni_feats_va = cv_uni_va.fit_transform(unigram_pos).toarray()
pos_bi_feats_va = cv_bi_va.fit_transform(bigrams_pos).toarray()
pos_tri_feats_va = cv_tri_va.fit_transform(trigram_pos).toarray()

In [139]:
binary_labels_tr = np.array(preprocessing.create_labels(labels=tr_dict['label'],label_values={'false':1, 'true':-1,'pants-fire':1,'barely-true':1,'half-true':0,'mostly-true':-1}))
binary_labels_va = np.array(preprocessing.create_labels(labels=va_dict['label'],label_values={'false':1, 'true':-1,'pants-fire':1,'barely-true':1,'half-true':0,'mostly-true':-1}))

In [140]:
from sklearn.preprocessing import Binarizer
tr_indexes = [i for i,x in enumerate(binary_labels_tr) if x!=0]
va_indexes = [i for i,x in enumerate(binary_labels_va) if x!=0]
transformer = Binarizer().fit(pos_uni_feats[tr_indexes]) # fit does nothing.
X_train = transformer.transform(pos_uni_feats[tr_indexes])
transformer = Binarizer().fit(pos_uni_feats[va_indexes])
X_test =  transformer.transform(pos_uni_feats[va_indexes])
y_train = binary_labels_tr[tr_indexes]
y_test = binary_labels_va[va_indexes]

In [141]:
m_clf = DecisionTreeClassifier(max_depth=5,criterion='entropy')
m_clf.fit(X_train, y_train)
predicted = m_clf.predict(X_test)
print(m_clf.score(X=X_test, y=y_test))
print(cross_val_score(m_clf, X_test, y_test, cv=10))

0.5492277992277992
[0.58653846 0.51923077 0.53846154 0.54807692 0.55769231 0.58653846
 0.63106796 0.5631068  0.59223301 0.54368932]


In [124]:
## train bigram_pos features on logistic regression
LR = LogisticRegression()
LR = LR.fit(X=X_train, y=y_train)
y_pred = LR.predict(X_test)
print("accuracy metrics for logistic regression classifier:\n",classification_report(y_test, y_pred))

accuracy metrics for logistic regression classifier:
              precision    recall  f1-score   support

         -1       0.40      0.27      0.32       420
          1       0.59      0.73      0.65       616

avg / total       0.52      0.54      0.52      1036



In [125]:
### train features on multinomial naive bayes classifier
nb_cly=MultinomialNB()
nb_cly.fit(X=X_train, y=y_train)
y_pred=nb_cly.predict(X_test)
print("accuracy metrics for Multinomial naive bayes classifier:\n",classification_report(y_test, y_pred))

accuracy metrics for Multinomial naive bayes classifier:
              precision    recall  f1-score   support

         -1       0.38      0.16      0.23       420
          1       0.59      0.81      0.68       616

avg / total       0.50      0.55      0.50      1036



In [126]:
### train features on SVM

svm_clf=SVC(C=1, kernel='rbf', degree=3, gamma='auto', random_state=None)
svm_clf=svm_clf.fit(X=X_train, y=y_train)
y_pred=svm_clf.predict(X_test)
print("accuracy metrics for support vector machine classifier:\n",classification_report(y_test, y_pred))

accuracy metrics for support vector machine classifier:
              precision    recall  f1-score   support

         -1       0.41      0.27      0.33       420
          1       0.60      0.74      0.66       616

avg / total       0.52      0.55      0.52      1036



In [100]:
### train features on SGD
sgd_clf = linear_model.SGDClassifier(max_iter=1000)
sgd_clf=sgd_clf.fit(X=X_train, y=y_train)
y_pred=sgd_clf.predict(X_test)
print("accuracy metrics for stochastic gardient desent classifier:\n",classification_report(y_test, y_pred))

accuracy metrics for stochastic gardient desent classifier:
              precision    recall  f1-score   support

         -1       0.37      0.14      0.20       420
          1       0.59      0.84      0.69       616

avg / total       0.50      0.56      0.49      1036



In [101]:
### train features on random forest
randFor_clf=RandomForestClassifier(n_estimators=2, criterion='gini', max_features='auto')
randFor_clf=randFor_clf.fit(X=X_train, y=y_train)
y_pred=randFor_clf.predict(X_test)
print("accuracy metrics for random forest classifier:\n",classification_report(y_test, y_pred))

accuracy metrics for random forest classifier:
              precision    recall  f1-score   support

         -1       0.41      0.41      0.41       420
          1       0.60      0.60      0.60       616

avg / total       0.52      0.52      0.52      1036



In [ ]:


## set up stanfordcorenlp server
nlp = StanfordCoreNLP('http://localhost:9000')

# Importing the dataset
dataset=pd.read_csv('train.tsv',delimiter='\t',encoding='utf-8')
dataset.columns=['statement_ID','label','statement','subject','speaker','job_title',
           'state_info','pantry_affiliation','barely_true_cnt','false_cnt',
           'half_true_cnt','mostly_true_cnt','pants_on_fire_cnt','context']


dataset.head()

total_rows = len(dataset.index)
cols = list(dataset.columns.values)

# Cleaning the texts
corpus = []
for i in range(0, total_rows):
    review = re.sub('[^a-zA-Z]', ' ', dataset['statement'][i])
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    corpus.append(review)

train_data=corpus
## define key-value
data_dict=dict()

data_dict['ids'] = train_data[['ID']].values[:,0]
data_dict['labels'] = train_data[['Label']].values[:,0]
data_dict['statements'] = train_data[['Statement']].values[:,0]
data_dict['subjects'] = train_data[['Subject']].values[:,0]
data_dict['contexts'] = train_data[['Context']].values[:,0]
# Process each sentence
# Obtain the Part Of Speech for every word in the statement
# Get POS bigram way
# Get POS trigram way

word_pos_list = list()
pos_list = list()
unigram_pos_list=list()
bigram_pos_list = list()
trigram_pos_list = list()

for sent_id, txt in enumerate(data_dict['text']):
    print('Procesing sentence number {0}: {1}'.format(sent_id,txt))
    output = nlp.annotate(txt, properties={
      'annotators': 'tokenize,pos,parse',
      'outputFormat': 'json'
      })
    result_word_pos = str()
    result_pos = list()
    unigram_pos=list()
    bigram_pos = list()
    trigram_pos = list()
    for o in output['sentences']:
        result_pos.append('<s>')
        for t in o['tokens']:
            result_word_pos += '{0}/{1} '.format(t['word'],t['pos'])
            result_pos.append('{0}'.format(t['pos']))
        for rpIndex, rp in enumerate(result_pos):
            if rpIndex==len(result_pos):
                unigram_pos.append('{0}').format(rp, result_pos[rpIndex+1])
            if rpIndex < len(result_pos)-1 and len(result_pos)>=2:
                bigram_pos.append('{0} {1}'.format(rp, result_pos[rpIndex+1]))
            if rpIndex < len(result_pos)-2 and len(result_pos)>=3:
                trigram_pos.append('{0} {1} {2}'.format(rp, result_pos[rpIndex+1], result_pos[rpIndex+2]))

    word_pos_list.append(result_word_pos)
    pos_list.append(result_pos)
    unigram_pos_list.append(unigram_pos)
    bigram_pos_list.append(bigram_pos)
    trigram_pos_list.append(trigram_pos)

##
data_dict['word_pos'] = word_pos_list
data_dict['pos'] = pos_list
data_dict['unigram'] = unigram_pos_list
data_dict['bigrams'] = bigram_pos_list
data_dict['trigrams'] = trigram_pos_list

##
dataset['Word POS'] = data_dict['word_pos']
dataset['POS'] = data_dict['pos']
dataset['unigram_pos'] = data_dict['unigram']
dataset['bigrams_pos'] = data_dict['bigrams']
dataset['trigram_pos'] = data_dict['trigrams']

## use countVectorizor to build postag output as features for training ML model
cv = CountVectorizer()

pos_uni_feats = cv.fit_transform(dataset['unigram_pos']).toarray()
pos_big_feats = cv.fit_transform(dataset['bigram_pos']).toarray()
pos_trig_feats = cv.fit_transform(dataset['trigram_pos']).toarray()

X_train, X_test, y_train, y_test  = train_test_split(pos_uni_feats,
        dataset['class'],train_size=0.8, random_state=123)

## train bigram_pos features on logistic regression
LR = LogisticRegression()
LR = LR.fit(X=X_train, y=y_train)
y_pred = LR.predict(X_test)
print("accuracy metrics for logistic regression classifier:\n",classification_report(y_test, y_pred))

### train features on multinomial naive bayes classifier
nb_cly=OneVsRestClassifier(MultinomialNB()).fit(X=X_train, y=y_train)
y_pred=nb_cly.predict(X_test)
print("accuracy metrics for Multinomial naive bayes classifier:\n",classification_report(y_test, y_pred))

### train features on SVM

svm_clf=SVC(C=1, kernel='rbf', degree=3, gamma='auto', random_state=None)
svm_clf=svm_clf.fit(X=X_train, y=y_train)
y_pred=svm_clf.predict(X_test)
print("accuracy metrics for support vector machine classifier:\n",classification_report(y_test, y_pred))

### train features on SGD
sgd_clf = linear_model.SGDClassifier(max_iter=1000)
sgd_clf=sgd_clf.fit(X=X_train, y=y_train)
y_pred=sgd_clf.predict(X_test)
print("accuracy metrics for stochastic gardient desent classifier:\n",classification_report(y_test, y_pred))

### train features on random forest
randFor_clf=RandomForestClassifier(n_estimators=2, criterion='gini', max_features='auto', class_weight={1:.9, 2:.5, 3:.01})
randFor_clf=randFor_clf.fit(X=X_train, y=y_train)
y_pred=randFor_clf.predict(X_test)
print("accuracy metrics for random forest classifier:\n",classification_report(y_test, y_pred))


##----------- More compact solution for training several emprical ML classfier model -----------##
### use kfold cross validation
## Kforld cross validation on training set
kf = KFold(len(X_train), numFolds=10, shuffle=True)

X_train, X_test, y_train, y_test  = train_test_split(pos_uni_feats,
        dataset['class'],train_size=0.8, random_state=123)
Y=dataset['class']

params = [{}, {'loss': 'log', 'penalty': 'l2', 'n_iter':1000,class_weight:{1:.9, 2:.5, 3:.01}}]
Models = [LogisticRegression, SGDClassifier, MultinomialNB, RandomForestClassifier, SVC]

for param, Model in zip(params, Models):
    total = 0
    for train_indices, test_indices in kf:
        train_X = X_train[train_indices, :]; train_Y = y_train[train_indices]
        test_X = X_test[test_indices, :]; test_Y = y_test[test_indices]
        reg = Model(**param)
        reg.fit(train_X, train_Y)
        predictions = reg.predict(test_X)
        total += accuracy_score(test_Y, predictions)

    accuracy = total / numFolds
    print("Accuracy score of {0}: {1}: {2}: {3}: {4}".format(Model.__name__, accuracy))
    print("accuracy metrics for {0}: {1}: {2}: {3}: {4}".format(Model.__name__, classification_report(y_test, predictions)))